In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling.models import custom_model
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.modeling.fitting import (_validate_model,
                                      _fitter_to_model_params,
                                      _model_to_fit_params, Fitter,
                                      _convert_input)
from astropy.modeling.optimizers import Simplex
import batman
from astropy.modeling import models, fitting

http://docs.astropy.org/en/stable/modeling/new.html

http://docs.astropy.org/en/stable/modeling/new.html#using-a-custom-statistic-function

https://en.wikipedia.org/wiki/Linear_least_squares_(mathematics)

In [6]:
#First, create the user-defined statistic

def chi_line(measured_vals, updated_model, x_sigma, y_sigma, x):
    
    """
    Chi^2 statistic for fitting a straight line with uncertainties in x and
    y.

    Parameters
    ----------
    measured_vals : array
    updated_model : `~astropy.modeling.ParametricModel`
        model with parameters set by the current iteration of the optimizer
    x_sigma : array
        uncertainties in x
    y_sigma : array
        uncertainties in y
    """
    
    model_vals = updated_model(x)
    if x_sigma is None and y_sigma is None:
        return np.sum((model_vals - measured_vals) ** 2)
    elif x_sigma is not None and y_sigma is not None:
        weights = 1 / (y_sigma ** 2 + updated_model.parameters[1] ** 2 *
                       x_sigma ** 2)
        return np.sum((weights * (model_vals - measured_vals)) ** 2)
    else:
        if x_sigma is not None:
            weights = 1 / x_sigma ** 2
        else:
            weights = 1 / y_sigma ** 2
        return np.sum((weights * (model_vals - measured_vals)) ** 2)

In [7]:
#Next, define the fitting optimizer

class LineFitter(Fitter):
    """
    Fit a straight line with uncertainties in both variables

    Parameters
    ----------
    optimizer : class or callable
        one of the classes in optimizers.py (default: Simplex)
    """

    def __init__(self, optimizer=Simplex):
        self.statistic = chi_line
        super().__init__(optimizer, statistic=self.statistic)

In [20]:
def __call__(self, model, x, y, x_sigma=None, y_sigma=None, **kwargs):
    """
    Fit data to this model.

    Parameters
    ----------
    model : `~astropy.modeling.core.ParametricModel`
        model to fit to x, y
    x : array
        input coordinates
    y : array
        input coordinates
    x_sigma : array
        uncertainties in x
    y_sigma : array
        uncertainties in y
    kwargs : dict
        optional keyword arguments to be passed to the optimizer

    Returns
    ------
    model_copy : `~astropy.modeling.core.ParametricModel`
        a copy of the input model with parameters set by the fitter

    """
    model_copy = _validate_model(model,
                                 self._opt_method.supported_constraints)

    farg = _convert_input(x, y)
    farg = (model_copy, x_sigma, y_sigma) + farg
    p0, _ = _model_to_fit_params(model_copy)

    fitparams, self.fit_info = self._opt_method(
        self.objective_function, p0, farg, **kwargs)
    _fitter_to_model_params(model_copy, fitparams)

    return model_copy

In [21]:
def BATMAN_MODEL(Rp, Baseline, t = None):
    #Next we create a TransitParams object to store the physical parameters describing the transit:

    params = batman.TransitParams()
    params.t0 = 0.                       #time of inferior conjunction
    params.per = 39.09432                #period in hours
    params.rp = Rp                       #planet radius (in units of stellar radii)
    params.a = 16.                       #semi-major axis (in units of stellar radii)
    params.inc = 90.                     #orbital inclination (in degrees)
    params.ecc = 0.                      #eccentricity
    params.w = 90.                       #longitude of periastron (in degrees)
    params.u = [0.1956, 0.3700]          #limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"       #limb darkening model
        
    m = batman.TransitModel(params, t)    #initializes model
    
    flux = m.light_curve(params)*Baseline        #calculates light curve
    return flux

In [22]:
x = np.linspace(0,100,101)
y = BATMAN_MODEL(Rp=0.02,Baseline=1,t=x)

In [16]:
#__call__(self,model=BATMAN_MODEL,x=x,y=y)

TypeError: __call__() missing 1 required positional argument: 'self'

In [23]:
s = LineFitter()
s.__init__()

TypeError: Can't instantiate abstract class LineFitter with abstract methods __call__